In [1]:
"""
Purpose: To run spine detection on a generic mesh

"""

'\nPurpose: To run spine detection on a generic mesh\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/neurd_packages/NEURD/neurd")

In [7]:
from datasci_tools import numpy_dep as np
np.array([np.array([1,2,3,4,5])]).dtype

dtype('int64')

# Load the data

In [ ]:
import trimesh
import numpy as np
from datasci_tools import numpy_utils as nu
from mesh_tools import trimesh_utils as tu

In [ ]:
from pathlib import Path
list(Path("./").iterdir())

In [ ]:
file_base = "full"
verts_file = f"{file_base}_vert.txt"
tri_file = f"{file_base}_tri.txt"
tri_array = nu.loadtxt(str(tri_file),dtype = "int",delimiter = " ")
verts_array =  nu.loadtxt(str(verts_file),dtype = "float",delimiter = " ")

In [ ]:
mesh = tu.mesh_from_vertices_faces(vertices = verts_array, faces = tri_array)
mesh

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    mesh,
    buffer = 0
)

In [ ]:
import ipyvolume as ipv
import numpy as np
N = 1000
x, y, z = np.random.normal(0, 1, (3, N))

fig = ipv.figure()
scatter = ipv.scatter(x, y, z)
ipv.show()

# Decimating

In [ ]:
mesh_dec = tu.decimate(mesh,decimation_ratio=0.25,)
mesh_dec = tu.largest_conn_comp(mesh_dec)
ipvu.plot_objects(
    mesh_dec,
    buffer=0
)


# Doing the spine extraction

In [ ]:
import spine_utils as spu

In [ ]:
# --- shaft parameters ---
spu.soma_vertex_nullification_global = False
spu.skeleton_endpoint_nullification = False

spu.clusters_threshold_global = 6
spu.smoothness_threshold_global = 0.08
spu.shaft_mesh_volume_max_global = 0.4
spu.shaft_close_hole_area_top_2_mean_max_global = 0.4
spu.shaft_mesh_n_faces_min_global = 200


# spine filtering parameters
spu.spine_n_face_threshold_bare_min_global = 310
spu.spine_sk_length_threshold_bare_min_global = 0.6
spu.filter_by_volume_threshold_bare_min_global = 0.011
spu.bbox_oriented_side_max_min_bare_min_global = 0.4
spu.sdf_mean_min_bare_min_global = 0.1
spu.spine_volume_to_spine_area_min_bare_min_global = 0.00002

# head neck segmentation
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 400

spine_objs = spu.spine_objs_bare_minimum_filt_with_attr_from_branch_obj(
    mesh = mesh_dec,
    plot_filtered_spines = True,
)

In [ ]:
spine_objs

In [ ]:
connectivity = "vertices"
min_n_faces_conn_comp = 0
mesh_filtered = mesh_dec
conn_mesh,conn_faces = tu.split_significant_pieces(mesh_filtered,
                                significance_threshold=min_n_faces_conn_comp,
                               return_face_indices=True,
                               connectivity=connectivity)

In [ ]:
conn_faces[0].dtype

In [ ]:
x = np.array([np.array([4,5,6,7])],dtype='object')

In [ ]:
x[0].dtype

In [ ]:
"""
Things to fix:

1) pip3 install -e not working
2) ipyvolume widgets not working
3) debug the array
"""


# Inspecting the Spines

In [ ]:
nviz.plot_objects(spine_objs[10].mesh)

In [ ]:
spine_objs[0].export()

In [ ]:
hdju.proofreading_neurons_table

# Exporting the figure

In [ ]:
spu.plot_spines_objs_with_head_neck_and_coordinates(
    spine_objs,
    mesh = mesh_dec,
)

In [ ]:
import ipyvolume_utils as ipvu
ipvu.save_to_html(f"{file_base}_spine_detection.html")

# Skeleton Mappings

In [ ]:
import skeleton_utils as sk
#skeletonize_and_clean_connected_branch_CGAL

In [ ]:
skeleton_surface = sk.surface_skeleton(
    mesh_dec,
    plot = True,
    #n_surface_downsampling=3,
)

In [ ]:
nviz.plot_objects(
    skeletons=[skeleton,sk_cgal],
    skeletons_colors=["red","blue"],
    buffer=0
)

In [ ]:
sk_removed

In [ ]:
skeleton = sk.skeleton_cgal(
    mesh_dec,
    plot = True,
    cgal_original_parameters = False,
    max_triangle_angle =1.91986,
    quality_speed_tradeoff=0.1,
    medially_centered_speed_tradeoff=0.2,#0.2,
    area_variation_factor=0.0001,
    max_iterations=500,#500,
    is_medially_centered=True,
    min_edge_length = 1,
    edge_length_multiplier = 0.002,
)

skeleton_no_cycles = sk.remove_cycles_from_skeleton(skeleton,verbose=True,)

sk_cgal,sk_removed = sk.clean_skeleton(
    skeleton_no_cycles,
    min_distance_to_junction = 3,
    return_removed_skeletons = True,
)

nviz.plot_objects(
    mesh_dec,
    skeletons=[sk_cleaned,sk.stack_skeletons(sk_removed)],
    skeletons_colors=["blue","red"],
    buffer=0
)

In [ ]:
# calculating the skeleton endpoints


In [ ]:
spu.spine_objs_with_border_sk_endpoint_and_soma_filter_from_scratch_on_mesh(
    mesh = mesh_dec,
)

In [ ]:
clusters = 6
smoothness = 0.08


# --- determining the shaft ---
spu.shaft_mesh_volume_max_global = 0.7
spu.shaft_close_hole_area_top_2_mean_max_global = 0.6
spu.shaft_mesh_n_faces_min_global = 200

sp_objs = spu.spine_objs_with_border_sk_endpoint_and_soma_filter_from_scratch_on_branch_obj(
    mesh = mesh_dec,
    skeleton = sk_cgal,
    clusters = clusters,
    smoothness = smoothness,
    soma_vertex_nullification = False,
    plot_segmentation=True,
    plot_shaft=True,
    plot = True
)

In [ ]:
clusters = 6
smoothness = 0.08


# --- determining the shaft ---
spu.shaft_mesh_volume_max_global = 0.7
spu.shaft_close_hole_area_top_2_mean_max_global = 0.6
spu.shaft_mesh_n_faces_min_global = 200

nviz.plot_objects(
    mesh_dec,
    meshes = [k.mesh for k in Out[82]],
    meshes_colors="red",
    buffer = 0
)

In [ ]:
sp_objs_with_att = spu.calculate_spine_attributes_for_list(
        sp_objs,
        calculate_coordinates=False,
)

sp_objs_with_att = [spu.calculate_spine_obj_mesh_skeleton_coordinates(
    spine_obj = k,
    mesh = mesh_dec,
) for k in sp_objs_with_att]

In [ ]:
spine_df = spu.df_from_spine_objs(sp_objs_with_att,add_volume_to_area_ratio=True)

In [ ]:
spine_df.columns

In [ ]:
columns = [
    "skeletal_length",
    "n_faces",
    "volume",
    "sdf_mean",
    "bbox_oriented_side_max",
    "spine_volume_to_spine_area",
]

spine_df[columns]

In [ ]:
not_spines = [30,]
big_spine_would_like_split_by_smoothness = [
    0,
    7,
    9,
    
]
big_spines_with_self_touches = [
    1,
    2,
    4, # self touch and some that could be separated
    5,
]
spine_with_end_on_it = [3]
double_head = [6,11,13]
big_single_head = [8]

In [ ]:
print(spu.spine_n_face_threshold_bare_min_global,
spu.spine_sk_length_threshold_bare_min_global,
spu.filter_by_volume_threshold_bare_min_global,
spu.bbox_oriented_side_max_min_bare_min_global,
spu.sdf_mean_min_bare_min_global,
spu.spine_volume_to_spine_area_min_bare_min_global,)

In [ ]:

# --- shaft parameters ---
spu.shaft_mesh_volume_max_global = 0.7
spu.shaft_close_hole_area_top_2_mean_max_global = 0.6
spu.shaft_mesh_n_faces_min_global = 200

# spine filtering parameters
spu.spine_n_face_threshold_bare_min_global = 310
spu.spine_sk_length_threshold_bare_min_global = 0.6
spu.filter_by_volume_threshold_bare_min_global = 0.011
spu.bbox_oriented_side_max_min_bare_min_global = 0.4
spu.sdf_mean_min_bare_min_global = 0.1
spu.spine_volume_to_spine_area_min_bare_min_global = 0.00002

# head neck segmentation
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 400

In [ ]:
verbose = True
sp_objs_filt = spu.filter_spine_objs_by_size_bare_minimum(sp_objs_with_att)
if verbose:
    print(f"AFTER filtering len(sp_objs_filt) = {len(sp_objs_filt)}")

In [ ]:
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 400
sp_objs_filt_head_neck = spu.calculate_spine_attributes_for_list(sp_objs_filt,calculate_head_neck=True, mesh = mesh_dec)

In [ ]:
sp_obj = sp_objs_filt_head_neck[0]
sp_obj.head_mesh,sp_obj.n_heads

In [ ]:
spu.plot_spines_objs_with_head_neck_and_coordinates(
    sp_objs_filt_head_neck,
    mesh = mesh_dec,
)

In [ ]:
sdfs=[0.242122  0.2696905 0.394093  0.439196  0.733061 ], ray_trace_perc = [0.23194992 0.58525092 0.35166066 0.35188387 0.17948695], mesh_sizes = [ 607 4864 2035 1990  111]

In [ ]:
spu.head_smoothness_global

In [ ]:
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 600
spu.head_smoothness_global = 0.03
spu.spine_head_neck(
    sp_objs_filt[21].mesh,
    plot_segmentation = True,
    verbose = True,
    plot_head_neck = True,
    #only_allow_one_connected_component_neck = True
)

# Doing it by hand

In [ ]:
delete_temp_file = True
clusters = 6
smoothness = 0.08
delete_temp_file = True
plot_segmentation = True

current_mesh = mesh_dec

cgal_data,cgal_sdf_data = tu.mesh_segmentation(
    current_mesh,
    #cgal_folder=cgal_folder,
    clusters=clusters,
    smoothness=smoothness,
    return_sdf=True,
    delete_temp_files=delete_temp_file,
    return_meshes=False,
    return_ordered_by_size=False,
    plot_segmentation = plot_segmentation,
)

split_meshes,split_meshes_idx = tu.split_mesh_into_face_groups(current_mesh,cgal_data,return_idx=True,
                                   check_connect_comp = False)

In [ ]:
# import ipyvolume_utils as ipvu
# ipvu.plot_mesh(
#     split_meshes[110]
# )

In [ ]:
meshes = list(split_meshes.values())
functions = [
            "close_hole_area_top_2_mean",
            "n_faces",
            "mesh_volume"
        ]
stats_df = tu.stats_df(meshes,functions)
stats_df

In [ ]:
close_hole_area_top_2_mean_max = 0.6
mesh_volume_max = 0.7
n_faces_min = 200

In [ ]:
stats_df_filt = stats_df.query(f"n_faces > {n_faces_min}")
filt_meshes = [meshes[k] for k in stats_df_filt.index.to_list()]
nviz.plot_objects(
    mesh_dec,
    meshes = filt_meshes,
    meshes_colors="red",
    buffer=0
)

In [ ]:
close_hole_area_top_2_mean_max = spu.shaft_close_hole_area_top_2_mean_max_global
mesh_volume_max = spu.shaft_mesh_volume_max_global
n_faces_min = spu.shaft_mesh_n_faces_min_global

close_hole_area_top_2_mean_max = 1
mesh_volume_max = 1

In [ ]:
query = [
        f"(close_hole_area_top_2_mean > {close_hole_area_top_2_mean_max}) or (mesh_volume > {mesh_volume_max})",
        #f"(close_hole_area_top_2_mean > {close_hole_area_top_2_mean_max}) or (n_faces > {n_faces_max})",
        f"(n_faces > {n_faces_min})"
    ]

import pandas_utils as pu
query_str = pu.query_str(query,table_type="pandas")
stats_df.query(query_str)